In [ ]:
!wget https://www.kaggleusercontent.com/kf/69296115/eyJhbGciOiJkaXIiLCJlbmMiOiJBMTI4Q0JDLUhTMjU2In0..2sw3xPincbNmc84INNajwA.AetKwyArYXpOtRlecN67pEZu83QiNR0tA6qofiAw699iNXZQ6gHKNVHUDWMdadqGUkgHIc_emTeFqmnYUrWDo_Rz4zrm5ociwsJsKlTc339OyHjQUkHAdQmAQ1Bfr0jPQN-z41fS3SEfOmL3PVkdzViM2MPvGVLIhBI0R2rWQ2q8NcERnQOhO7wvMTVpgF5bTsXCOQJoqCEct850y0JcGxVCY0VNNL0ujgIkJ3lm7Rme-3lPXcFqqEpBuq-5T_THzrkeF-kUwhO_zNHwInScirRLmolf6abdAXgNFjZ5XHP0-9moCeTj-VCozJMUxeh88f12IBSNFyz_hFR_A-lBgtRN6KGHEQr67R8aHmoO6W8YQ5lIo85BhLbEVcrGcVEwhI6EjJNJ6mVX3g1ccQjrdz73SaKYdCnQkYaNeG2a6xvxWtJSEzzCKkEYUgYsZdx1nVEgUBBNtCXQ8C0FwOM_hO-LN8vB2JQlEn46MCAsZYo4fLRX0vZ6243awp52eiwMJfZMbrJFtB0ZU5mvyDeDFmT2jv0GGBnI_dn5nfzsTfTdLjTaO4RCbwu6ubC9drnksS9De7c17ZxyonbG0Y9Nf89hqrvL1_F5-909n83jDx-DZUF46hp4GTwFtrL2QiSh5WCgLlG9WpVVzwIh9eWoJg.SDARW07I4ceeZBvBVx-qNw/bert-base-uncased.bin
!wget https://www.kaggleusercontent.com/kf/69296115/eyJhbGciOiJkaXIiLCJlbmMiOiJBMTI4Q0JDLUhTMjU2In0..IdOHcvqmdebDOB7Bl3PeBw.pmY3LmIr39X5PmlaRVqNnPbcmsrCKmDtWGavnmZg06cyBmYiYeNXmynkifqkY1JtEtPIqiQkmM3oEKpb9px644_UpSwDxPtym9osY6HFKCMD5zgWpbGhIXDWUeVQ84PUt-T8biSvMJeDUqBRGwki7IAJGqgY_h_h7FUZnLrJ9dYrTUKOPd47t2W4kJxQU4GEGT-77CDU_i1NkdpE7ErCYRk_0KOlT3aFWYkX5d4g2S-kZN11zr2jBKTqKeYVDcHincquTfa8bEAoAGOnT-mL7Bahr3Tu60atZ02rh9ljSw8cl-3XuBeCzSG_U4vEtMQHupe1yZxq1YRWIseQREW1LcMDB4TSFU0OrI-h3d9ecniJt_Sf1buMEXgRAgYadxjQjtWB_8LzMq3LPUS34dbcrx4fMnpBXAA7jcoCAOrUUQKwfHhtIi3IuZFPB-e9rdNO8nYQQWvgq-ZfIsdSjN8IiKlZEIKN-9PYV0XoyWfYmUd7f5FB1nFAHbUZuope9qoDWHyUBn-aoq_66L5A_kZPXdFKHqp99BA9BoJeGUullV5NRgcqImIxxbmPZmfI4Z5jACo06ssx0xXB-VvuSzasggxs1xyz8r3SK_u2-mauK4bVX4UPCxn5ce_--8D5q-3uc9GfWmFNYFBoyOYKS0tI4g.OyrT-Vf2EasNIjEJ8r562w/meta.bin

In [ ]:
!pip install seqeval
!pip install transformers

In [ ]:
from seqeval.metrics import accuracy_score
from seqeval.metrics import classification_report, accuracy_score, f1_score
import torch.nn as nn

In [ ]:
import transformers
import torch
base_model_path = "bert-base-uncased"
tokenizer = transformers.AutoTokenizer.from_pretrained(
    base_model_path,
    do_lower_case = False
)
class EntityDataset:

    def __init__(self, texts, tags):
        self.texts = texts
        self.tags = tags

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, item):
        text = self.texts[item]
        tags = self.tags[item]

        ids = []
        target_tags = []

        for i, s in enumerate(text):
            inputs = tokenizer.encode(
                s,
                add_special_tokens=False
            )
            input_len = len(inputs)
            ids.extend(inputs)
            target_tags.extend([tags[i]] * input_len)

        ids = ids[: max_len - 2]
        target_tags = target_tags[: max_len - 2]

        ids = [101] + ids + [102]
        target_tags = [1] + target_tags + [1]

        mask = [1] * len(ids)
        token_type_ids = [0] * len(ids)

        padding_len = max_len - len(ids)

        ids = ids + ([0] * padding_len)
        mask = mask + ([0] * padding_len)
        token_type_ids = token_type_ids + ([0] * padding_len)
        target_tags = target_tags + ([1] * padding_len)

        return {
            "ids": torch.tensor(ids, dtype=torch.long),
            "mask": torch.tensor(mask, dtype=torch.long),
            "token_type_ids": torch.tensor(token_type_ids, dtype=torch.long),
            "target_tags": torch.tensor(target_tags, dtype=torch.long)
        }

In [ ]:
class EntityModel(nn.Module):

    def __init__(self, enc_tag):
        super(EntityModel, self).__init__()

        self.num_tag = len(enc_tag.classes_)

        self.config = transformers.AutoConfig.from_pretrained(base_model_path)
        self.config._num_labels = self.num_tag
        self.config.label2id = {k: v for k, v in zip(enc_tag.classes_, enc_tag.transform(enc_tag.classes_))}
        self.config.id2label = {k: v for k, v in zip(enc_tag.transform(enc_tag.classes_), enc_tag.classes_)}
        self.classifier = transformers.AutoModelForTokenClassification.from_config(self.config)

    def forward(self, ids, mask, token_type_ids, target_tags):
        output_1 = self.classifier(ids, attention_mask=mask, token_type_ids=token_type_ids, labels=target_tags)
        return output_1

In [ ]:
import json
import re


# JSON formatting functions
def convert_data_to_spacy(JSON_FilePath):
    training_data = []
    lines=[]
    with open(JSON_FilePath, 'r') as f:
        lines = f.readlines()

    for line in lines:
        data = json.loads(line)
        text = data['data']
        data_annotations = data['label']
        training_data.append((text, {"entities" : data_annotations}))
    return training_data


def trim_entity_spans(data: list) -> list:
    """Removes leading and trailing white spaces from entity spans.

    Args:
        data (list): The data to be cleaned in spaCy JSON format.

    Returns:
        list: The cleaned data.
    """
    invalid_span_tokens = re.compile(r'\s')

    cleaned_data = []
    for text, annotations in data:
        entities = annotations['entities']
        valid_entities = []
        for start, end, label in entities:
            valid_start = start
            valid_end = end
            while valid_start < len(text) and invalid_span_tokens.match(
                    text[valid_start]):
                valid_start += 1
            while valid_end > 1 and invalid_span_tokens.match(
                    text[valid_end - 1]):
                valid_end -= 1
            valid_entities.append([valid_start, valid_end, label])
        cleaned_data.append([text, {'entities': valid_entities}])
    return cleaned_data


def clean_entities(training_data):
    clean_data = []
    for text, annotation in training_data:

        entities = annotation.get('entities')
        entities_copy = entities.copy()

        # append entity only if it is longer than its overlapping entity
        i = 0
        for entity in entities_copy:
            j = 0
            for overlapping_entity in entities_copy:
                # Skip self
                if i != j:
                    e_start, e_end, oe_start, oe_end = entity[0], entity[1], \
                                                       overlapping_entity[0], overlapping_entity[1]
                    # Delete any entity that overlaps, keep if longer
                    if ((oe_start <= e_start <= oe_end) or (oe_end >= e_end >= oe_start)) \
                            and ((e_end - e_start) <= (oe_end - oe_start)):
                        entities.remove(entity)
                j += 1
            i += 1
        clean_data.append((text, {'entities': entities}))

    return clean_data


def process_data(df):

    enc_tag = preprocessing.LabelEncoder()
    all_ents = df['entities_mapped'].apply(pd.Series).stack().values
    enc_tag.fit(all_ents)
    sentences = list(df["clean_content"].str.split())
    tag = list(df["entities_mapped"].apply(enc_tag.transform))

    return sentences, tag, enc_tag

In [ ]:
def valid(model, testing_loader, device):

    model.eval()
    predictions_, true_labels_ = [], []
    with torch.no_grad():
        for _, data in enumerate(testing_loader, 0):
            input_mask = data['mask']
            targets = data['target_tags']
            for k, v in data.items():
                data[k] = v.to(device, dtype=torch.long)
            output = model(**data)
            loss, logits = output[:2]
            logits = logits.detach().cpu().numpy()
            logits = [list(p) for p in np.argmax(logits, axis=2)]
            input_mask = input_mask.to('cpu').numpy()
            label_ids = targets.to('cpu').numpy()
            for i,mask in enumerate(input_mask):
                temp_1 = [] # Real one
                temp_2 = [] # Predict one
                
                for j, m in enumerate(mask):
                    # Mark=0, meaning its a pad word, dont compare
                    if m:
                            temp_1.append(enc_tag.inverse_transform([label_ids[i][j]])[0])
                            temp_2.append(enc_tag.inverse_transform([logits[i][j]])[0])
                    else:
                        break                
                true_labels_.append(temp_1)
                predictions_.append(temp_2)

    return predictions_, true_labels_

In [ ]:
import pandas as pd
import numpy as np
import joblib
from sklearn import preprocessing
from sklearn import model_selection
import numpy as np
from nltk.tokenize import WhitespaceTokenizer

meta_data = joblib.load("/content/meta.bin")
enc_tag = meta_data["enc_tag"]

max_len = 512
datas = trim_entity_spans(convert_data_to_spacy("/content/knowledge only.jsonl"))
datas = clean_entities(data)
df_data = pd.DataFrame(columns=["clean_content", "entities_mapped"])
entities_mapped = []
clean_content = []
for data in datas:
    entities = data[1]["entities"]
    words = []
    labels = []
    content = data[0].split()
    span_generator = WhitespaceTokenizer().span_tokenize(data[0])
    spans = [span for span in span_generator]
    for word, span in zip(content, spans):
        words.append(word)
        found = False
        for entity in sorted(entities):
            ent_start = entity[0]
            ent_end = entity[1]
            ent_label = entity[2]
            if ent_start <= span[0] <= ent_end:
                labels.append(ent_label)
                found = True
                break
        if not found:
            labels.append("O")

    entities_mapped.append(labels)
    clean_content.append(words)

df_data["entities_mapped"] = entities_mapped
df_data["clean_content"] = clean_content
df_data["clean_content"] = df_data["clean_content"].apply(lambda x: " ".join(x))

sentences, tag, enc_tag = process_data(df_data)


In [ ]:
(train_sentences, test_sentences, train_tag, test_tag) = model_selection.train_test_split(
    sentences, tag, random_state = 42, test_size = 0.3)

valid_dataset = EntityDataset(
    texts=test_sentences, tags=test_tag)

valid_data_loader = torch.utils.data.DataLoader(
    valid_dataset, batch_size=1, num_workers=2)

device = torch.device("cpu")
model = EntityModel(enc_tag=enc_tag)
model.load_state_dict(torch.load('/content/bert-base-uncased.bin', map_location=torch.device('cpu')))
model.to(device)
predictions, true_labels = valid(model, valid_data_loader, device)

In [ ]:
from sklearn.metrics import classification_report, f1_score, accuracy_score
print("Accuracy score: %f"%(accuracy_score(np.concatenate(true_labels), np.concatenate(predictions))))
print(classification_report(np.concatenate(true_labels), np.concatenate(predictions)))

Accuracy score: 0.938555
              precision    recall  f1-score   support

   Knowledge       0.83      0.73      0.78      5320
           O       0.95      0.97      0.96     30891

    accuracy                           0.94     36211
   macro avg       0.89      0.85      0.87     36211
weighted avg       0.94      0.94      0.94     36211

